# Building slope suitablity layers for GRIDCERF


The following code was used to build the slope suitability layers for GRIDCERF. GRIDCERF does not provide the source data directly due to some license restrictions related for direct redistribution of the unaltered source data.  However, the following details the provenance associated with each source dataset and how they were processed.


## 1. Setup environment

### 1.1 Download GRIDCERF


Download the GRIDCERF package if you have not yet done so from here: https://doi.org/10.5281/zenodo.6601789. Please extract GRIDCERF inside the data directory of this repository as the paths in this notebook are set to that expectation.

### 1.2 Install GDAL



This application requires GDAL to be installed.  We will call GDAL directly from your command prompt or terminal, so please ensure that you can do so before running the following cells.  More information on how to install GDAL can be found here:  https://gdal.org/download.html


### 1.3 Import necessary Python packages

In [1]:
import os

import numpy as np
import rasterio


## 2. Configuration


In [ ]:
# get the parent directory path to where this notebook is currently stored
root_dir = os.path.dirname(os.getcwd())

# data directory in repository
data_dir = os.path.join(root_dir, "data")

# GRIDCERF data directory from downloaded archive
gridcerf_dir = os.path.join(data_dir, "gridcerf")

# GRIDCERF reference data directory
reference_dir = os.path.join(gridcerf_dir, "reference")

# GRIDCERF technology specific data directory
output_dir = os.path.join(gridcerf_dir, "technology_specific")

# GRIDCERF source data directory
source_dir = os.path.join(gridcerf_dir, "source", "technology_specific")

# template land mask raster
template_raster = os.path.join(reference_dir, "gridcerf_landmask.tif")

# downloaded file
srtm_epsg6380 = os.path.join(source_dir, 'SRTM90_V4_slope_percent_1km_epsg6350.tiff')

# reprojected and masked file to CERF's CRS and extent
srtm_albers = os.path.join(source_dir, 'SRTM90_V4_slope_percent_1km_albers.tif')

# exclusion layer for wind where <= 20 percent slope is unsuitable
slope_20per = os.path.join(tech_dir, 'gridcerf_srtm_slope_20pct_or_less.tif')

# exclusion layer for common where <= 12 percent slope is unsuitable
slope_12per = os.path.join(tech_dir, 'gridcerf_srtm_slope_12pct_or_less.tif')

# exclusion layer for common where <= 10 percent slope is unsuitable
slope_10per = os.path.join(tech_dir, 'gridcerf_srtm_slope_10pct_or_less.tif')

# exclusion layer for common where <= 5 percent slope is unsuitable
slope_5per = os.path.join(tech_dir, 'gridcerf_srtm_slope_5pct_or_less.tif')


## 3. Generate slope suitability rasters

- **Title**:  SRTM Digital Elevation Data Version 4
- **Description from Source**: The Shuttle Radar Topography Mission (SRTM) digital elevation dataset was originally produced to provide consistent, high-quality elevation data at near global scope. This version of the SRTM digital elevation data has been processed to fill data voids, and to facilitate its ease of use.
- **Source URL**:  https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4#description
- **Date Accessed**:  12/31/21
- **Citation**
> Jarvis, A., Reuter, H. I., Nelson, A. & Guevara, E. Hole-filled SRTM for the globe Version 4. CGIAR Consortium for Spatial Information https://research.utwente.nl/en/publications/hole-filled-srtm-for-the-globe-version-4-data-grid (2008).


### 3.1 Generate the following slope raster from SRTM 90m data via Google Earth Engine

```javascript

// get sample area to extract|
var geometry = 
    ee.Geometry.Polygon(
        [[[-128.59843750000002, 50.17140850495461],
          [-128.59843750000002, 23.55613021145419],
          [-64.78984375000002, 23.55613021145419],
          [-64.78984375000002, 50.17140850495461]]], null, false);

// DEM
var dataset = ee.Image('CGIAR/SRTM90_V4').clip(geometry);

// extract elevation data
var elevation = dataset.select('elevation');

// calculate slope in degrees
var slope = ee.Terrain.slope(elevation);

// convert to percent slope
var slope_percent = slope.divide(100).multiply(Math.PI).tan().multiply(100).rename('Percent');

// reproject into NAD83 Albers in meters; CERF's standard ESRI:102003 is not available
var percent_albers = slope_percent.reproject({
  crs: 'EPSG:6350',
  scale: 1000
});

// export the result to my drive
Export.image.toDrive({
  image: percent_albers,
  description: 'SRTM90_V4_slope_percent_1km_epsg6350',
  folder: 'earth_engine',
  region: geometry,
  scale: 1000,
  crs: 'EPSG:6350'
});
```

### 3.2 Reproject to GRIDCERF's CRS and extent

In [3]:
# gdal warp command
gdal_cmd = f'gdalwarp -s_srs EPSG:6350 -t_srs ESRI:102003 -tr 1000.0 1000.0 -r bilinear -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -te_srs ESRI:102003 -multi -of GTiff {srtm_epsg6380} {srtm_albers}'

os.system(gdal_cmd)


0

### 3.3 Reclassify to less than or equal to target percent slope as 0 (suitable) and above 1 (unsuitable) 

In [4]:
with rasterio.open(land_mask) as msk:
    mask_arr = msk.read(1)
    mask_arr = np.where(mask_arr == 0, np.nan, mask_arr)


In [23]:

# create 20 percent suitable raster
with rasterio.open(srtm_albers) as src:
    
    # update metadata datatype to int16
    metadata = src.meta.copy()
    metadata.update({'dtype': rasterio.int16})
    
    arr = src.read(1)
    
    # make all greater than 20 percent unsuitable
    arr = np.where(arr > 20, 1.0, 0.0)
    
    # apply land mask
    arr *= mask_arr
    
    # convert nan to unsuitable
    arr = np.where(np.isnan(arr), 1, arr)
    
    with rasterio.open(slope_20per, 'w', **metadata) as dest:
        dest.write(arr.astype(rasterio.int16), 1)


In [ ]:
# create 12 percent suitable raster
with rasterio.open(srtm_albers) as src:
    
    # update metadata datatype to int16
    metadata = src.meta.copy()
    metadata.update({'dtype': rasterio.int16})
    
    arr = src.read(1)
    
    # make all greater than 20 percent unsuitable
    arr = np.where(arr > 12, 1.0, 0.0)
    
    # apply land mask
    arr *= mask_arr
    
    # convert nan to unsuitable
    arr = np.where(np.isnan(arr), 1, arr)
    
    with rasterio.open(slope_12per, 'w', **metadata) as dest:
        dest.write(arr.astype(rasterio.int16), 1)


In [5]:
# create 10 percent suitable raster
with rasterio.open(srtm_albers) as src:
    
    # update metadata datatype to int16
    metadata = src.meta.copy()
    metadata.update({'dtype': rasterio.int16})
    
    arr = src.read(1)
    
    # make all greater than 10 percent unsuitable
    arr = np.where(arr > 10, 1.0, 0.0)
    
    # apply land mask
    arr *= mask_arr
    
    # convert nan to unsuitable
    arr = np.where(np.isnan(arr), 1, arr)
    
    with rasterio.open(slope_10per, 'w', **metadata) as dest:
        dest.write(arr.astype(rasterio.int16), 1)


In [6]:
# create 5 percent suitable raster
with rasterio.open(srtm_albers) as src:
    
    # update metadata datatype to int16
    metadata = src.meta.copy()
    metadata.update({'dtype': rasterio.int16})
    
    arr = src.read(1)
    
    # make all greater than 5 percent unsuitable
    arr = np.where(arr > 5, 1.0, 0.0)
    
    # apply land mask
    arr *= mask_arr
    
    # convert nan to unsuitable
    arr = np.where(np.isnan(arr), 1, arr)
    
    with rasterio.open(slope_5per, 'w', **metadata) as dest:
        dest.write(arr.astype(rasterio.int16), 1)
        